# Line Parameters Estimation

## Haversine distance between two nodes

The distance between nodes is calculated using the haversine formula. Haversine formula calculates the great-circle distance between two points, i.e. the shortest distance over the Earth's surface. The haversine formula is given as follows.

$\begin{align}
&\text{\underline{Haversine formula}} \notag \\
&a = \sin^2(\Delta\frac{\phi}{2}) + \cos \phi_1 \cos \phi_2 \sin^2(\Delta\frac{\lambda}{2}) \notag\\
&c = 2 \times \text{atan2}(\sqrt{a},\sqrt{1-a}) \notag\\
&distance = R \times c \notag\\
&\text{where } \phi \text{ is latitude (rad), } \lambda \text{ is longitude (rad), and } R \text{ is the Earth radius (6,371km).} \notag
\end{align}$

## Reactance and resistance approximation

Given the distance between nodes, i.e. the estimated length of candidates, we can now approximate the reactance of the AC line and the resistance of the HVDC link in terms of length based on the following references [],[].

The reactance of the AC line, $x_{ij}$, can be approximated using the following equation.

$\begin{align}
&x_{ij} = L \times x^{pos-seq} \notag\\
&\text{where } L \text{ is the length of AC line } x_{ij} \text{ (km)}\notag\\
&\text{ and } x^{pos-seq} \text{ is the positive-sequence reactance, which is typically }0.327\Omega/\text{km}
\end{align}$

Furthermore, the resistance of the HVDC link, $r_{ij}$, can be approximated using the following equation.

$\begin{align}
&r_{ij} = L \times r^{pos-seq} \notag\\
&\text{where } L \text{ is the length of HVDC link } r_{ij} \text{ (km)}\notag\\
&\text{ and } r^{pos-seq} \text{ is the positive-sequence resistance, which is typically }0.0378\Omega/\text{km}
\end{align}$

We convert the line reactance and link resistance to their per-unit equivalents. Base voltage of 400kV and base power of 100MVA are used according to []. The base impedance can be calculated as follows.

$\begin{align}
z_{base} = \frac{v_{base}^2}{s_{base}} = \frac{400kV^2}{100MVA} = 1600\Omega
\end{align}$

Thus, the per-unit reactance and resistance can be calculated accordingly.

$\begin{align}
z_{pu} = \frac{z}{z_{base}} = \frac{r + jx}{z_{base}}
\end{align}$

For AC line, $ z = jx $, and for HVDC link $ z = r $.

## Load package

In [1]:
using CSV, DataFrames
using Combinatorics
include(joinpath("..","src","utils.jl"))

getFullCandidateLines

## Load data

In [2]:
# network data
nodes = CSV.read(joinpath("..","data","NordicBaltic","nodes.csv"),DataFrame)
generators = CSV.read(joinpath("..","data","NordicBaltic","generators.csv"),DataFrame)
first(generators,10)

Row,id,name_id,name,node,node_name,carrier,type,pmax,encost,rucost,rdcost,efficiency,pu_max,pu_min,area
,Int64,String31,String,Int64,String15,String15,String15,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,1,Esbjerg1_GAS,Esbjerg Reserve power station,1,Denmark,Gas,Gas,25.0,118.93,29.7325,29.7325,0.35,1,0.0,1
2,2,Grenaa1_GAS,Grenaa Reserve power station,1,Denmark,Gas,Gas,25.0,118.93,29.7325,29.7325,0.35,1,0.0,1
3,3,IRU_GAS,IRU Elektrijaam power station,2,Estonia,Gas,Gas,173.0,92.5,23.125,23.125,0.45,1,0.0,2
4,4,Lietuvos1_GAS,Lietuvos power station,5,Lithuania,Gas,Gas,300.0,118.93,29.7325,29.7325,0.35,1,0.0,2
5,5,Lietuvos2_GAS,Lietuvos power station,5,Lithuania,Gas,Gas,300.0,118.93,29.7325,29.7325,0.35,1,0.0,2
6,6,Lietuvos3_GAS,Lietuvos power station,5,Lithuania,Gas,Gas,445.0,75.68,18.92,18.92,0.55,1,0.0,2
7,7,Fyns7_GAS,Fyns power station,1,Denmark,Gas,Gas,362.0,75.7,18.925,18.925,0.55,1,0.0,1
8,8,Hammerfest_GAS,Hammerfest Snohvit Terminal power station,7,Norway,Gas,Gas,229.0,75.7,18.925,18.925,0.55,1,0.0,1
9,9,Hillerød1_GAS_CHP,Hillerød power station,1,Denmark,Gas,Gas CHP,75.0,81.5,20.375,20.375,0.51,1,0.0,1


## Get full candidate HVAC lines
- Only connection within the same area is permitted -> ignore_area=false

In [3]:
fullcac = getFullCandidateLines(nodes,DataFrame();ignore_area=false,hvdc=false,kv_nom=400)

Row,id,f_node,t_node,f_area,t_area,length,x
,Int64,Any,Any,Int64,Int64,Float64,Float64
1,1,1,3,1,1,1167.18,0.238543
2,2,1,6,1,1,296.174,0.0605306
3,3,1,7,1,1,544.403,0.111262
4,4,3,6,1,1,925.963,0.189244
5,5,3,7,1,1,971.048,0.198458
6,6,6,7,1,1,605.283,0.123705
7,7,2,4,2,2,184.043,0.0376137
8,8,2,5,2,2,495.579,0.101284
9,9,4,5,2,2,322.054,0.0658197


## Get full candidate HVDC lines

In [4]:
fullcdc = getFullCandidateLines(nodes,DataFrame();ignore_area=true,hvdc=true,kv_nom=400)

Row,id,f_node,t_node,f_area,t_area,length,r
,Int64,Int64,Int64,Int64,Int64,Float64,Float64
1,1,1,2,1,2,1006.15,0.0237702
2,2,1,3,1,1,1167.18,0.0275747
3,3,1,4,1,2,982.022,0.0232003
4,4,1,5,1,2,885.507,0.0209201
5,5,1,6,1,1,296.174,0.00699712
6,6,1,7,1,1,544.403,0.0128615
7,7,2,3,2,1,368.546,0.00870689
8,8,2,4,2,2,184.043,0.004348
9,9,2,5,2,2,495.579,0.0117081


## Display the name of nodes and areas for reporting

### Candidate HVAC lines

In [5]:
tablefullcac = copy(fullcac)
tablefullcac.f_node = nodes[tablefullcac.f_node,:name]
tablefullcac.t_node = nodes[tablefullcac.t_node,:name]
tablefullcac.f_area = nodes[tablefullcac.f_area,:name]
tablefullcac.t_area = nodes[tablefullcac.t_area,:name]
tablefullcac

Row,id,f_node,t_node,f_area,t_area,length,x
,Int64,String15,String15,String15,String15,Float64,Float64
1,1,Denmark,Finland,Denmark,Denmark,1167.18,0.238543
2,2,Denmark,Sweden,Denmark,Denmark,296.174,0.0605306
3,3,Denmark,Norway,Denmark,Denmark,544.403,0.111262
4,4,Finland,Sweden,Denmark,Denmark,925.963,0.189244
5,5,Finland,Norway,Denmark,Denmark,971.048,0.198458
6,6,Sweden,Norway,Denmark,Denmark,605.283,0.123705
7,7,Estonia,Latvia,Estonia,Estonia,184.043,0.0376137
8,8,Estonia,Lithuania,Estonia,Estonia,495.579,0.101284
9,9,Latvia,Lithuania,Estonia,Estonia,322.054,0.0658197


### Candidate HVDC lines

In [6]:
tablefullcdc = copy(fullcdc)
tablefullcdc.f_node = nodes[tablefullcdc.f_node,:name]
tablefullcdc.t_node = nodes[tablefullcdc.t_node,:name]
tablefullcdc.f_area = nodes[tablefullcdc.f_area,:name]
tablefullcdc.t_area = nodes[tablefullcdc.t_area,:name]
tablefullcdc

Row,id,f_node,t_node,f_area,t_area,length,r
,Int64,String15,String15,String15,String15,Float64,Float64
1,1,Denmark,Estonia,Denmark,Estonia,1006.15,0.0237702
2,2,Denmark,Finland,Denmark,Denmark,1167.18,0.0275747
3,3,Denmark,Latvia,Denmark,Estonia,982.022,0.0232003
4,4,Denmark,Lithuania,Denmark,Estonia,885.507,0.0209201
5,5,Denmark,Sweden,Denmark,Denmark,296.174,0.00699712
6,6,Denmark,Norway,Denmark,Denmark,544.403,0.0128615
7,7,Estonia,Finland,Estonia,Denmark,368.546,0.00870689
8,8,Estonia,Latvia,Estonia,Estonia,184.043,0.004348
9,9,Estonia,Lithuania,Estonia,Estonia,495.579,0.0117081


## Export data

In [7]:
CSV.write(joinpath("..","data","NordicBaltic","fullcac.csv"),fullcac)
CSV.write(joinpath("..","data","NordicBaltic","fullcdc.csv"),fullcdc)

"../data/NordicBaltic/fullcdc.csv"